In [1]:
import ase.units as au
import numpy as np
import os
import shutil


In [4]:
class DensityOptimizer():
    
    def __init__(self):
        pass
    
    def initialize(self, workdir, dt, mu):
        self.workdir = workdir
        #dens = self.read_density(dens_file)
        #self.density = self.normalize(dens)
        #self.X = np.sqrt(self.density)
        
        #grad = self.read_gradient(grad_file)
        #self.dEdX = self.normalize(grad)
        
        self.dt = dt
        self.mu = mu
        
    def calculate_dEdX(self, density_file):
        """
        calculates dEdX for the density specified in density file
        """
        # copy density file to work_dir/density
        shutil.copy(density_file, os.path.join(self.workdir, 'density'))
        # run PROFESS-1
        self.run_profess()
        
    def calculate_lambda(self):
        Ne = 12.0
        N_p = (self.X_p*self.X).sum()
        N_pp = (self.X_p*self.X_p).sum()
        tau = self.dt**2/self.mu
        p2 = N_p/(tau*Ne)
        q = (N_pp - Ne)/(tau**2*Ne)
        lambda_1 = -p2 - np.sqrt(p2**2 - q)
        lambda_2 = -p2 + np.sqrt(p2**2 - q)
        return(lambda_1, lambda_2)
        
    def normalize(self, dens):
        """
        multiply density with dV per voxel
        """
        V = (3.9691/au.Bohr)**3
        dV = V/16**3
        density_vox = dens*dV
        return(density_vox)
    
    def optimize(self, nsteps, density_file = None, overwrite = False):
        for i in range(nsteps):
            
            # create path to density file
            density_file = os.path.join(self.workdir, f'density_{i}')
#             if density_file == None:
#                 density_file = os.path.join(self.workdir, f'density_{i}')
#             else:
                
            # calculate gradient for density file
            self.calculate_dEdX(density_file)
            
            # read gradient and density into python
            grad_file = os.path.join(self.workdir, 'dEdX')
            grad = self.read_gradient(grad_file)
            self.dEdX = self.normalize(grad)
            dens = self.read_density(density_file)
            self.density = self.normalize(dens)
            self.X = np.sqrt(self.density)

            # propagate density
            if i == 0:
                # first step
                a = -self.dEdX/self.mu
                self.X_p = a*self.dt**2 + self.X
            else:
                self.X_p = self.vv_step()
            
            # enforce that number of electrons is conserved
            lambda_1, lambda_2 = self.calculate_lambda()
            self.X_p = self.X_p + self.dt**2/self.mu*lambda_2*self.X_p
            
            # write new density to file
            density_p = np.power(self.X_p,2)
            if overwrite:
                self.save_density(density_p, os.path.join(self.workdir, f'density'))
            else:
                self.save_density(density_p, os.path.join(self.workdir, f'density_{i+1}'))
            
            # update sqrt of density for previous step
            self.X_m = self.X
            #self.X = self.X_p
    
    def read_density(self, dens_file):
        with open(dens_file, 'r') as f:
            file = f.readlines()
            density_unparsed = file[0][108:]
            density = density_unparsed.split()
            density = np.array([float(i) for i in density])
            #density = density.reshape((16,16,16))
            return(density)

    def read_gradient(self, filep):
        # load potential
        dEdX = []
        with open(filep, 'r') as f:
            for i, line in enumerate(f):
                if i != 0:
                    dEdX.append(float(line.strip('\n')))
        dEdX = np.array(dEdX)
        #dEdX = dEdX.reshape((16, 16, 16))
        return(dEdX)
    
    def run_profess(self):
        os.chdir(self.workdir)
        p = subprocess.run(['/home/misa/software/PROFESS-1/PROFESS', os.path.join(self.workdir, 'job.inpt')], capture_output = True,  text=True )
        return(p)
    
    def save_density(self, density):
        dens_str = '  x-dimension:          16   y-dimension:          16   z-dimension:          16   # of spins:            1 '
        for d in density:
            dens_str += "{:.20E} ".format(d)

        new_dens = '/home/misa/software/PROFESS-1/calculate_gradient/new_density'
        with open(new_dens, 'w') as f:
            f.write(dens_str)

    def vv_step(self):
        X_p = 2*self.X - self.X_m - (self.dt**2/self.mu)*self.dEdX
        return(X_p)
    

    

In [5]:
OptimizeTest = DensityOptimizer()
OptimizeTest.initialize('/home/misa/software/PROFESS-1/calculate_gradient', 0.1, 1)

In [ ]:
density_path = '/home/misa/software/PROFESS-1/calculate_gradient/density_0'
gradient_path = '/home/misa/software/PROFESS-1/calculate_gradient/dEdX_0'
#density_m_path = '/home/misa/software/PROFESS-1/calculate_gradient/density_0'

mu = 1
dt = 0.1

In [ ]:
density = read_density(density_path)
density_normalized = normalize(density)
X = np.sqrt(density_normalized)

gradient_not_normalized = read_gradient(gradient_path)
dEdX = normalize(gradient)

In [ ]:
if step == 0:
    # propagate density  first step
    a = -dEdX/mu
    X_p = a*dt**2 + X
else:
    read_density()

### generate start density
- run PROFESS for one step
- print density -> initial guess
- see /home/misa/git_repositories/PROFESS/test/single_iteration

### call profess-1
- copy intial density
- call profess-1
- do it from python

### propagate density
- load density and gradient into python
- reshape
- propagate (with velocity verlet)

In [2]:
def load_density(dens_file):
    with open(dens_file, 'r') as f:
        file = f.readlines()
        density_unparsed = file[0][108:]
        density = density_unparsed.split()
        density = np.array([float(i) for i in density])
        density = density.reshape((16,16,16))
        return(density)

In [3]:
density = load_density('/home/misa/software/PROFESS-1/calculate_gradient/density_0')
density2 = load_density('/home/misa/software/PROFESS-1/calculate_gradient/new_density_test')
np.array_equal(density, density2)

True

In [4]:
# load gradient
filep = '/home/misa/software/PROFESS-1/calculate_gradient/dEdX'
dEdX = []
with open(filep, 'r') as f:
    for i, line in enumerate(f):
        if i != 0:
            dEdX.append(float(line.strip('\n')))
dEdX = np.array(dEdX)
dEdX = dEdX.reshape((16, 16, 16))

In [5]:
# what are the units of density and dE/dX?
import ase.units as au
V = (3.9691/au.Bohr)**3
dV = V/16**3
density_vox = density*dV
dEdX_vox = dEdX*dV

In [6]:
print((density*dV).sum())
print("Density must be converted to Voxel/Bohr^3")

11.999999866902229
Density must be converted to Voxel/Bohr^3


#### What are appropiate values for mu and dt?
- $\mu = 1$ and $dt = 0.1$ (both in a.u.) (10.1103/physrevlett.55.2471)

In [ ]:
mu = 1
dt = 0.1

In [7]:
# propagate density  first step
a = -dEdX_vox/mu
x_1 = a*dt**2 + np.sqrt(density_vox)

In [ ]:
# propagate density velocity verlet
def vv_step(X, X_m, dt, mu, dEdX):
    X_p = 2*X - X_m - (dt**2/mu)*dEdX
    return(X_p)

# load density from previous step
dens_prev = load_density('/home/misa/software/PROFESS-1/calculate_gradient/density_0')
dens_prev = dens_prev*dV
X_minus = np.sqrt(dens_prev)
X = np.sqrt(density_vox)

In [ ]:
x_1 = vv_step(X, X_minus, dt, mu, dEdX_vox)

### enforce constraint

In [8]:
Ne = 12.0
N_p = (x_1*np.sqrt(density_vox)).sum()
N_pp = (x_1*x_1).sum()
tau = dt**2/mu

In [9]:
p2 = N_p/(tau*Ne)
q = (N_pp - Ne)/(tau**2*Ne)
lambda_2 = -p2 + np.sqrt(p2**2 - q)

In [10]:
x_1_constr = x_1 + dt**2/mu*lambda_2*x_1
density_1 = np.power(x_1_constr,2)

In [11]:
density_1.sum()

11.999908874690611

### save new density

In [12]:
start_str = '  x-dimension:          16   y-dimension:          16   z-dimension:          16   # of spins:            1 '
density_1_fl = density_1.flatten()/dV
for d in density_1_fl:
    start_str += "{:.20E} ".format(d)
    
new_dens = '/home/misa/software/PROFESS-1/calculate_gradient/new_density'
with open(new_dens, 'w') as f:
    f.write(start_str)

In [ ]:
# start_str = '  x-dimension:          16   y-dimension:          16   z-dimension:          16   # of spins:            1 '
# density_1_fl = density.flatten()
# length = len(start_str)
# for d in density_1_fl:
#     start_str += "{:.20E} ".format(d)
#     length += len(str(d)) + 1
    
# new_dens = '/home/misa/software/PROFESS-1/calculate_gradient/new_density_test'
# with open(new_dens, 'w') as f:
#     f.write(start_str)
#     #f.write('\n')

In [ ]:
#len('0.62344209078022891357E-02')
#len('3.04456931547079749645E-02')